## sequence prediction

In [1]:
import tensorflow as tf
from tensorflow import nn
import numpy as np
import math

lstm_size  = 512
n_layers   = 3
n_epochs   = 100 
seq_len    = 50
batch_size = 50

### utils

In [ ]:
def xavier(n_in, n_out):
    init_range = 4*math.sqrt(6.0/(n_in + n_out))
    return tf.random_uniform([n_in, n_out], -init_range, init_range)

### data loading

In [2]:
with open('shakespeare.txt') as f:
    words_full = [c for c in list(f.read())]
    vocab = []
    for c in words_full:
        if c not in vocab:
            vocab.append(c)
    vocab_size = len(vocab)    
    
    i_split = int(len(words_full)*0.8)
    words_tr = np.array([vocab.index(el) for el in words_full[:i_split]], dtype='int32')
    words_te = np.array([vocab.index(el) for el in words_full[i_split:]], dtype='int32')

    #make batches
    tr_x = words_tr.copy()
    #target is just data shifted one element to the right
    tr_y = np.roll(words_tr, 1)
    sp = [i*batch_size*seq_len for i in range(1,len(words_tr)//(batch_size*seq_len) + 1)]
    n_batches = len(tr_x)//(batch_size*seq_len)
    if(len(tr_x) % batch_size*seq_len != 0):
        # drop last small batch
        tr_x = np.array(np.split(tr_x, sp)[:-1]).reshape(n_batches, seq_len, batch_size)
        tr_y = np.array(np.split(tr_y, sp)[:-1]).reshape(n_batches, seq_len, batch_size)
    #c = np.zeros((10, len(vocab)))
    #c[np.arange(10), words_tr[:10]]=1

### model

In [25]:
#create model
tf.reset_default_graph()

lstm = nn.rnn_cell.BasicLSTMCell(lstm_size)
stacked_lstm = nn.rnn_cell.MultiRNNCell([lstm] * n_layers)
input_data = tf.placeholder(tf.int32, [batch_size, seq_len])
targets = tf.placeholder(tf.int32, [batch_size, seq_len])
initial_state = state = stacked_lstm.zero_state(batch_size, tf.float32)

with tf.device("/cpu:0"):
    embedding = tf.get_variable("embedding", [vocab_size, lstm_size])
    inputs = tf.nn.embedding_lookup(embedding, input_data)

for i in range(seq_len):
    if i > 0:
        tf.get_variable_scope().reuse_variables()
    output, state = stacked_lstm(inputs[:,i,:], state)
final_state = state

softmax_w = tf.Variable([lstm_size, len(vocab)], dtype=tf.float32)
softmax_b =  tf.Variable([len(vocab)], dtype=tf.float32)

z = tf.matmul(output, softmax_w)
probs = tf.softmax(z)
loss = tf.nn.seq2seq.sequence_loss_by_example([z], [tf.reshape(targets, [-1])], [tf.ones([batch_size * seq_len])] )
cost = tf.reduce_sum(loss) / batch_size


ValueError: Shape (2,) must have rank 2

### training

In [20]:
init = tf.initialize_all_variables()
#train
with tf.Session() as s:
    s.run(init)
    for i in range(n_epochs):
        for j in range(n_batches):
            print('Epoch {}, batch {}'.format(i, j))
            s.run([output, final_state], feed_dict={input_data:tr_x[j]})


Epoch 0, batch 0
Epoch 0, batch 1
Epoch 0, batch 2
Epoch 0, batch 3
Epoch 0, batch 4
Epoch 0, batch 5
Epoch 0, batch 6


KeyboardInterrupt: 